In [ ]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [ ]:
import numpy as np

In [ ]:
host = 'fireteam108c-db.ctq6oauek6z5.us-east-2.rds.amazonaws.com:3306'
user = 'admin108c'
password = 'fireteam108c'
database = 'amazon_books'

In [ ]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

In [ ]:
connection = engine.connect()

In [ ]:
books_rating = pd.read_sql_table('books_rating', con=connection)
display(books_rating)

In [ ]:
books_rating['rating'] = books_rating['rating'].astype(float)

In [ ]:
books_ratings = pd.read_sql_table('books_rating_raw', con=connection)
books_ratings

In [ ]:
mask = books_ratings["User_id"].notna() & books_ratings["review/score"].notna() & books_ratings["Title"].notna()
books_ratings = books_ratings.loc[mask]
books_ratings["Title"].unique().shape[0]

In [ ]:
books_ratings = books_ratings.drop_duplicates().drop(columns=["Price"])

In [ ]:
books_ratings

In [ ]:
books_id = pd.read_csv('book_dict_keys.csv')
books_id.drop_duplicates("Title", inplace=True)

In [ ]:
books_id.dropna(inplace=True)

In [ ]:
books_data

In [ ]:
books_data = books_data.merge(books_id, on="Title", how="inner").drop_duplicates("Title")
books_data.to_sql('books_data', con=connection, if_exists='replace', index=False)

In [ ]:
books_ratings.drop_duplicates(["Title", "User_id", "review/summary", "review/text"], inplace=True)
books_ratings.columns

In [ ]:
reviewed_books = books_data[books_data["Title"].isin(books_ratings["Title"])]
reviewed_books.to_sql('reviewed_books', con=connection, if_exists='replace', index=False)

In [ ]:
books_ratings = books_ratings.merge(reviewed_books, on="Title", how="inner")

In [ ]:
books_ratings.to_sql('ratings_user_tagged', con=connection, if_exists='replace', index=False, chunksize=10000)

In [ ]:
connection.rollback()

In [ ]:
reviewed_tagged_books = pd.read_sql_table('books_data', con=connection)

In [ ]:
ratings_user_tagged = pd.read_sql_table('ratings_user_tagged', con=connection)

In [ ]:
ratings_user_tagged

In [ ]:
res = { k: v.get(["book_id", "review/score"]) for k, v in ratings_user_tagged.groupby('User_id').filter(lambda x: len(x) > 10).groupby('User_id')}

In [ ]:
matrix = pd.DataFrame(0.0, index=res.keys(), columns = reviewed_tagged_books["book_id"].values)
matrix

In [ ]:
for k, v in res.items():
	t = v.reset_index(drop=True).transpose()
	matrix.loc[k][t.loc["book_id"]] = t.loc["review/score"]
# 4 min 20 sec runtime


In [14]:
matrix.to_parquet('matrix.parquet.gzip',
              compression='gzip')

In [ ]:
matrix.to_csv("matrix.csv")

In [ ]:
test = pd.read_csv("matrix.csv", index_col=0)
test

In [ ]:
connection.close()